This is the final version of the code. It is intendet to include everything we have.

In [ ]:
import glob
import time
import eyed3
import librosa
import numpy as np
import pandas as pd

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('wordnet')


import string as s
import re

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score,accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')

## Will take a lifetime to run

In [ ]:
cols = "title artist album genre mood y sr chroma_mean chroma_std mfccs_mean mfccs_std centroid_mean centroid_std centroid_skew contrast_mean contrast_std roll_off_mean roll_off_std roll_off_skew zero_mean zero_std zero_skew tempo beat".split()
df = pd.DataFrame([], columns=cols)

sid = SentimentIntensityAnalyzer()

start_time = time.process_time()

#BEWARE that this function will retrieve all of the items in the directory including
#files that are not folders. In this usage case, it should only be used where the "dataset" directory
#includes genre files.

#Our file structure:

    #-dataset (root)
        #-genre1
            #genre_lyrics.csv
            #song1
            #...
            #song-1
        #-genre2
            #genre_lyrics.csv
            #song1
            #...
            #song-1
        #...
        #-genre-1
            #genre_lyrics.csv
            #song1
            #...
            #song-1
            
            
genres = glob.glob("*") #In dataset folder here
song_count = 1
for genre in genres:
    lyrics_df = pd.read_csv(genre+genre.strip("\\")+"_lyrics.csv", names=["lyrics"]) #Reaing lyrics
    print(lyrics_df)
    songs = glob.glob(genre+"*.mp3") #All songs

    for song_path in songs:
        audiofile = eyed3.load(song_path)
        print("Start processing song #{song_cnt}: {song_name}".format(song_cnt=song_count, song_name=audiofile.title))
        y, sr = librosa.load(song_path)

            #Chroma
        chroma=librosa.feature.chroma_cens(y=y, sr=sr)
        chroma_mean=np.mean(chroma,axis=1)
        chroma_std=np.std(chroma,axis=1)

            #MFCCS
        mfccs=librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfccs_mean=np.mean(mfccs,axis=1)
        mfccs_std=np.std(mfccs,axis=1)

            #Centroid
        centroid=librosa.feature.spectral_centroid(y=y, sr=sr)
        centroid_mean=np.mean(centroid,axis=1)
        centroid_std=np.std(centroid,axis=1)
        centroid_skew=stats.skew(centroid,axis=1)[0]

            #Contrast
        contrast=librosa.feature.spectral_contrast(y=y, sr=sr)
        contrast_mean=np.mean(contrast,axis=1)
        contrast_std=np.std(contrast,axis=1)

            #Roll off
        roll_off=librosa.feature.spectral_rolloff(y=y, sr=sr)
        roll_off_mean=np.mean(roll_off,axis=1)
        roll_off_std=np.std(roll_off,axis=1)
        roll_off_skew=stats.skew(roll_off,axis=1)[0]

            #Zero Crossing Rate
        zero=librosa.feature.zero_crossing_rate(y)
        zero_mean=np.mean(zero,axis=1)
        zero_std=np.std(zero,axis=1)
        zero_skew=stats.skew(zero,axis=1)[0]

            #Tempo Beat
        tempo, beat = librosa.beat.beat_track(y=y, sr=sr)

            #Mood (NLP)
        if(sid.polarity_scores(lyrics_df["lyrics"][song_count-1])["pos"] - sid.polarity_scores(lyrical)["neg"] > 0.05):
            mood = 1
        else:
            mood = -1

        df = df.append({"title": audiofile.tag.title, "artist": audiofile.tag.artist, "album": audiofile.tag.album,
                        "genre": audiofile.tag.genre, "y": y,
                        "chroma_mean": chroma_mean, "chroma_std": chroma_std,
                        "mfccs_mean": mfccs_mean, "mfccs_std": mfccs_std,
                        "centroid_mean": centroid_mean, "centroid_std": centroid_std, "centroid_skew": centroid_skew,
                        "contrast_mean": contrast_mean, "contrast_std": contrast_std,
                        "roll_off_mean": roll_off_mean, "roll_off_std": roll_off_std, "roll_off_skew": roll_off_skew,
                        "zero_mean": zero_mean, "zero_std": zero_std, "zero_skew": zero_skew,
                        "tempo": tempo, "beat": beat,
                        "mood": mood}, ignore_index=True)

        song_count+=1
    
    df.to_csv("data.csv")
print("The whole process took", time.process_time() - start_time, "seconds.")